### Capstone Idea

[GitHub](https://github.com/biborsz/Capstone)

**Problem Statement:** 

Beta.SAM.gov is the successor, among other federal governmental websites, of fedbizops.gov. The part that interests me is that it has a searchable collection of federal contract solicitations awards, as well as an information system of past grants. The current system is not yet fully operational. Nonetheless, it allows to search the collection based on key terms, grant id, etc. One of it's problems in the past was, that while it was searchable, it did not have a recommender system. Businesses, especially small businesses had to spend a considerable amount of time to find relevant grant opportunities. This new system offers that. Creating an account will have the advantage of receiving updates. However, the downside of email updates is that it clogs up the email inbox, and it does not always generate the expected value:
- it is difficult to know whether there are more opportunities out there or just the ones about which a business gets notified
- reader fatigue may cause businesses not to look further for contract opportunities than what had already been sent to them - and thus leave potential fundings on the table
- someone still needs to sift through a large amount of potentially irrelevant federal grant descriptions

All in all: a user controlled recommender system would enhance the effectiveness of grant searching for businesses that do not have lots of resources to allocate toward that activity in the first place. 

How it would work:

- based on archival data - it would search for similarities in ## not possible - archived data does not contain this information
   - business activity of applicants
   - name of funding agency
   
   - earlier search terms
   - successful earlier grant applications

-> it would give a list of potentially useful search terms, with an option to click on a select few

-> based on the search terms selected - it will further tune the recommender system

-> businesses have an option of rating a recommendation up or down

- based on earlier search terms - one would have access to a longer list of opportunites - that would make browsing possible (right now - it looks to me - that is out of the question)

- companies could search for other relevant information - for example: which companies are applying for similar grants in their business and geographic area
   
   
[API documentation](https://open.gsa.gov/api/get-opportunities-public-api/#user-account-api-key-creation)

[Beta.SAM.gov](https://beta.sam.gov/)

**Methodology:**
   - content based recommender system
   - text vectorizer: *bag-of-words*, *one-hot-encoder* 
     - bag of words: extracts words from the corpus as features
     - one-hot-encoder: gives a value of one if it finds a feature within a row/ text (0 to all other features)
   - classifies/ recommends based on *cosine similarities*

**Sources:**

https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d

https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html

http://recommender-systems.org/content-based-filtering/

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

**Ideas for the future:**
 - find database of registered businesses (business activity/ line of business)
 - crossreference awardees in prior grants to provide a list of competitors

In [1]:
# imports
import pandas as pd
import numpy as np
import requests
import time
import datetime

In [2]:
# set display options 
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

In [ ]:
# rewrite code - to read in combined file - 
# separete out dates 
# split string '-'
# turn them into datetime object
# check for oldest date
# turn back into string
# make that postedTo date
# have for loop pull another page
# transform json object
# concat it to existing dataframe
# save it to csv
# I am at 03/02/2020

In [3]:
# define function to return minimum postedDate as string
def get_min_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).min()
    return min_date.strftime('%m/%d/%Y')

In [4]:
# define function to return maximum postedDate as string
def get_max_post_date(filename):
    # read in file of downloaded contract opportunities
    df = pd.read_csv(filename)
    # get min posted date
    min_date = pd.to_datetime(df['postedDate']).max()
    return min_date.strftime('%m/%d/%Y')

In [5]:
# define function to return today's date as string
def today():
    now = datetime.datetime.now()
    return now.strftime('%m/%d/%Y')

In [6]:
today()

'05/29/2020'

In [4]:
get_max_post_date('./data/combined.csv')

'05/14/2020'

In [7]:
# pull contract information from api.sam.gov

# set base url
url = 'https://api.sam.gov/prod/opportunities/v1/search'

# create empty list to store results
result = []
# initialize counter
count = 0
# downloaded contract opportunities - file name
# file = './data/combined.csv'

# set postedTo date to today's date by calling today() function
postedTo = today()

# set posted from date
postedFrom = get_max_post_date('./data/combined.csv')

# for loop to pull contracts
for i in range(4):
    count += 1
    
    # do a get request
    req = requests.get(url,
                      params={
                          'api_key': '',
                          'postedFrom': postedFrom,
                          'postedTo': postedTo,
                          'limit': 1000,
                          'offset': (count-1) * 1000
                      })
    
    # add response to result list
    result.append(req)
    
    now = datetime.datetime.now()
    print('Time:', now.strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(5)
    
    
    
# source for datetime - https://www.w3resource.com/python-exercises/python-basic-exercise-3.php

Time: 2020-05-29 14:53:47
Time: 2020-05-29 14:54:18
Time: 2020-05-29 14:54:48
Time: 2020-05-29 14:55:05


In [8]:
# unpack list of json objects from response data
ops = []
for item in result:
    print(item.headers)
    ops.append(item.json())

{'Age': '25', 'Content-Type': 'application/hal+json', 'Date': 'Fri, 29 May 2020 18:53:47 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.55.40, 10.177.55.40', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Set-Cookie': 'citrix_ns_id=LmIrIOAeke8D9noZKI9SNWLoQ8I0002; Domain=.sam.gov; Path=/; Secure; HttpOnly', 'Cache-Control': 'private', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked'}
{'Age': '26', 'Content-Type': 'application/hal+json', 'Date': 'Fri, 29 May 2020 18:54:18 GMT', 'Server': 'openresty', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'Via': 'http/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Cache': 'MISS', 'X-Forwarded-For': '74.96.156.35, 10.177.16.72, 10.177.52.13, 1

In [9]:
# parse json objects
ls_data = []
for i in range(len(ops)):
    print(ops[i].keys())
    df = pd.DataFrame(ops[i]['opportunitiesData'])
    ls_data.append(df)
    data = pd.concat(ls_data)

dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])
dict_keys(['totalRecords', 'limit', 'offset', 'opportunitiesData', 'links'])


In [10]:
data.shape

(3532, 27)

In [11]:
data.tail(10)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,archiveDate,typeOfSetAsideDescription,typeOfSetAside,responseDeadLine,naicsCode,classificationCode,active,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
522,026e9523c1c048b5a61d1c59914e1720,Amendment 0003 DDWG Tension Fabric Structures Overhead Doors Replacement,SP330020Q0061,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA DISTRIBUTION,2020-05-27,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,auto15,2020-06-24,Total Small Business Set-Aside (FAR 19.5),SBA,2020-06-09T15:30:00-04:00,332311,J099,Yes,None,"[{'fax': '', 'type': 'primary', 'email': 'tameka.drayton@dla.mil', 'phone': '7177706523', 'title': None, 'fullName': 'Tameka L. Drayton'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=026e9523c1c048b5a61d1c59914e1720,OFFICE,"{'zipcode': '17070-5008', 'city': 'NEW CUMBERLAND', 'countryCode': 'USA', 'state': 'PA'}","{'city': {'code': '80508', 'name': 'Warner Robins'}, 'state': {'code': 'GA', 'name': 'Georgia'}, 'zip': '31098', 'country': {'code': 'USA', 'name': 'UNITED STATES'}}",None,https://beta.sam.gov/opp/026e9523c1c048b5a61d1c59914e1720/view,"[{'rel': 'self', 'href': 'https://api.sam.gov/prod/opportunities/v1/search?noticeid=026e9523c1c048b5a61d1c59914e1720&limit=1'}]","[https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/8ea4b24d158045eb98d9943044c476cd/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/431966cce4684be89be9695260570529/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/5a3eef34e9e048528cd9b08bb0c01ef1/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/057e1ff64dda4c5488370d5c025eec93/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/85a36af3eff243faa7789932c0446098/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/db003c83501a4d22b89d92cfb9cc7047/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/18a30f8b10f7476e8c2ae87f555ee6d9/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/e53f6fedc7f74f3e816ab72cc3ad9492/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/7be690a096514fe69adf4603e8d68478/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/a8178e3840644402a752fd147190dd70/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/219cfa51a0114a549b01a35615379de1/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/2a98a1f8d28b4f5e86a2eed7cfa5f0c1/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/cd5bef2b182b41d48d1f42e44ca558de/download?api_key=null&token=, https://beta.sam.gov/api/prod/opps/v3/opportunities/resources/files/395f1fad998b4dd6b5f3e26797b629dc/download?api_key=null&token=]"
523,0265256f47264de8b12634ffad11c53c,Personnel Safety System Conduit Systems and Cables Installation,RFP20-068C,"ENERGY, DEPARTMENT OF","ENERGY, DEPARTMENT OF",PRINCETON LAB - DOE CONTRACTOR,2020-05-27,Solicitation,Solicitation,autocustom,2020-07-31,None,None,2020-06-30T16:00:00-04:00,238210,Z1HA,Yes,None,"[{'fax': '6092432021', 'type': 'primary', 'email': 'ccheung@pppl.gov', 'phone': '6092433092', 'title': None, 'fullName': 'Chi Man Cheung'}]",https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=0265256f47264de8b12634ffad11c53c,OFFICE,"{'zipcode': '08540', 'city': 'Princeton', 'countryCode': 'USA', 'state': 'NJ'}","{'city': {'code': '60900', 'name': 'Princeton'}, 'state': {'code': 'NJ', 'name': 'New Jersey'}, 'zip': '08543', 'country': {'

In [12]:
data.duplicated('noticeId').sum()

0

In [15]:
# data.drop_duplicates('noticeId', inplace=True)

In [13]:
data.to_csv('./data/5_29_pull.csv', index=False)

In [14]:
df1 = pd.read_csv('./data/combined.csv')
df2 = pd.read_csv('./data/5_29_pull.csv')

In [17]:
df2.shape

(10000, 27)

In [15]:
df = pd.concat([df1, df2])

In [16]:
df.shape

(43454, 27)

In [17]:
df.to_csv('./data/combined.csv', index=False)